In [1]:
import matplotlib.pyplot as plt
import numpy as np
import mpmath as mp
mp.mp.dps = 50 #higher precision
from scipy.optimize import curve_fit
import scipy.integrate as integrate
from plotDensity import *
import json

In [2]:
def get_derivative_wrt_e(walker_results, precision=10):
    derivatives = []
    for result in walker_results:
        derivative = {}
        keys = sorted(result.keys())
        for i in range(1, len(keys)):
            x1, x2 = keys[i - 1], keys[i]
            y1, y2 = result[x1], result[x2]
            # Compute the derivative
            deriv = (y2 - y1) / (x2 - x1)
            # store derivative for left bound of tupel
            derivative[x1] = round(deriv, precision)
        derivatives.append(derivative)
    return derivatives

In [3]:
def cut_overlapping_histogram_parts(interval_data, stitching_keys):
    for i in range(len(stitching_keys)):
        stitching_energy_of_interval_i = stitching_keys[i]

        # Modify the i-th interval
        current_interval = interval_data[i]
        # Keep only keys <= stitching_energy_of_interval_i
        current_interval = {k: v for k, v in current_interval.items() if k <= stitching_energy_of_interval_i}

        # Modify the (i+1)-th interval if following interval is still in bounds
        if i + 1 < len(interval_data):
            next_interval = interval_data[i + 1]
            # Keep only keys > stitching_energy_of_interval_i
            next_interval = {k: v for k, v in next_interval.items() if k > stitching_energy_of_interval_i}

        # Update the intervals in the original list
        interval_data[i] = current_interval
        if i + 1 < len(interval_data):
            interval_data[i + 1] = next_interval

In [4]:
def log_sum_exp(to_sum):
    maxval = max(to_sum)
    exp_sum = 0
    for value in to_sum:
        exp_sum += mp.exp(value-maxval)
    res = maxval + mp.log(exp_sum)
    return res


def free_energy(E_list, log_g_list,  T):
    #Need to log sum over g(E)*exp(-E/T) without overflow issues
    to_sum = []
    for i, log_g in enumerate(log_g_list):
        to_sum.append(log_g - E_list[i]/T)
    maxval = max(to_sum)
    exp_sum = 0
    for value in to_sum:
        exp_sum += mp.exp(value-maxval)
    res = maxval + mp.log(exp_sum)
    return -T*res

def get_free_energies(rescaled_results,temperatures):
    free_energies = []
    for seed_results in rescaled_results:
        free_energy_classes = []
        for error_result in seed_results:
            f_values = []
            for T in temperatures:
                f_values.append(free_energy(error_result[0], error_result[1], T)/(-T))
            free_energy_classes.append(f_values)
        free_energies.append(free_energy_classes)
    return free_energies


def process_results(batch_results,X,Y):
    rescaled_results = []
    for seed_results in batch_results:

        rescaled_seed_results = []
        for error_result in seed_results:

            walker_results = error_result
            walker_results = get_renormalized_log_g_values_as_dict_list(walker_results)
            walker_results = average_matching_keys(walker_results)
            results_x = []
            results_y = []
            for result in walker_results:
                results_y.append(np.array(list(result.values())))
                results_x.append(np.array(list(result.keys())))

            derivatives_wrt_e = get_derivative_wrt_e(walker_results)
            minimum_deviation_energies = find_lowest_inverse_temp_deviation(derivatives_wrt_e)
            rescale_results_for_concatenation(results_x, results_y, minimum_deviation_energies)

            x_max = -1 -2*X*Y
            rescaled_x = []
            rescaled_y = []
            for i in range(len(results_x)):
                for j in range(len(results_x[i])):
                    if results_x[i][j] > x_max: #avoid double counting
                        x_max = results_x[i][j]
                        rescaled_x.append(results_x[i][j])
                        rescaled_y.append(results_y[i][j])

            offset = log_sum_exp(rescaled_y)
            rescaled_y = [res + mp.log(2)*X*Y - offset for res in rescaled_y]
            rescaled_seed_results.append([rescaled_x,rescaled_y])
        rescaled_results.append(rescaled_seed_results)
    return rescaled_results

def process_data(data, batch_results, p, size, error):
    for entry in data:
        histogram_seed = entry["histogram_seed"]
        run_seed = entry["run_seed"]
        results = entry["results"]

        E_list = []
        log_g_list = []

        # Process the results
        for key, value in results.items():
            E_list.append(int(key))
            log_g_list.append(float(value))

        batch_results.append({
                'prob': p,
                'size': size,
                'error': error,
                'histogram_seed': histogram_seed,
                'run_seed': run_seed,
                'E': E_list,
                'log_g': log_g_list
            })
        # offset =


In [5]:
def read_results_file(path):

    with open(path, 'r') as file:
        content = file.read()

    content = content.strip().rstrip(',')

    corrected_json = f'[{content}]'

    try:
        data = json.loads(corrected_json)
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")

    return data

def parse_file(filename):
    data = []
    try:
        with open(filename, 'r') as file:
            content = file.read()
            # print("file content printout (for debugging):")
            # print(content)
    except FileNotFoundError:
        return

    # Split content into individual blocks
    blocks = content.split('}\n{')  # Assuming blocks are separated by double newlines

    for block in blocks:
        # Extract histogram_seed, run_seed, and results
        histogram_seed_match = re.search(r'"histogram_seed": "(\d+)"', block)
        run_seed_match = re.search(r'"run_seed": "(\d+)"', block)
        results_match = re.search(r'"results": \[([^]]*)\]', block)

        if histogram_seed_match and run_seed_match and results_match:
            histogram_seed = histogram_seed_match.group(1)
            run_seed = run_seed_match.group(1)
            results_str = results_match.group(1)

            # Process results
            results = {}
            results_items = results_str.split(',')
            for item in results_items:
                key_value = item.split(':')
                if len(key_value) == 2:
                    key = key_value[0].strip().strip('"')
                    value = float(key_value[1].strip())
                    results[key] = value

            data.append({
                "histogram_seed": histogram_seed,
                "run_seed": run_seed,
                "results": results
            })

    return data


In [6]:
def get_seed_and_dicts(filename):
    with open(filename, 'r') as file:
        content = file.read()

    blocks = content.split("},\n")[:-1]

    all_results = {}
    for block in blocks:
        histogram_seed_match = int(re.search(r'"histogram_seed": "(\d+)"', block).group(1))
        run_seed_match = int(re.search(r'"run_seed": "(\d+)"', block).group(1))
        results_match = re.search(r'"results": \{([^}]*)\}', block).group(1)
        energy_blocks = results_match.split(',')

        energies = []
        log_g = []

        for e in energy_blocks:
            match = re.search(r'"(-?\d+)": (-?\d+\.\d{10})', e)
            energies.append(float(match.group(1)))
            log_g.append(float(match.group(2)))

        last_index = 0

        dict_list = []

        for i in range(1,len(energies)):
            if energies[i] < energies[i - 1]:
                dict_list.append(dict(zip(energies[last_index:i], log_g[last_index:i])))
                last_index = i
            if i==(len(energies)-1):
                dict_list.append(dict(zip(energies[last_index:], log_g[last_index:])))

        all_results[histogram_seed_match] = dict_list

    return all_results

In [8]:
boundary_type = "periodic"
probabilities = [0.1, 0.11, 0.12]
size = 4
intervals = 10
iterations = 1000
overlap = 0.25
walkers = 8
alpha = 0.8
beta = 1e-6
exchange = 50

for p in probabilities:
    for error in ["I", "X", "Y", "Z"]:

        filename = f"../results/periodic/prob_{p:.6f}/X_{size}_Y_{size}/error_class_{error}/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt"
        data = read_results_file(filename)

        diffs = []

        for d in data:
            diff = log_sum_exp(d['results'].values()) - mp.log(2)*4*4
            if np.abs(diff) > 1e-10:
                diffs.append((p, d['histogram_seed'], diff))

        print(diffs)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


### Find stitching_keys

In [23]:
seed = 1000
results_old = read_data_from_file(f"../results_without_post/periodic/prob_0.100000/X_4_Y_4/seed_{seed}/error_class_X/intervals_10_iterations_1000_overlap_0.250000_walkers_8_seed_run_42_alpha_0.800000_beta_0.0000010000exchange_offset50.txt")

"""averages over walker results per intervals"""
walker_results = average_matching_keys(results_old)

"""normalize the walker results by min value for log results"""
walker_results = get_renormalized_log_g_values_as_dict_list(walker_results)

In [24]:
derivatives = get_derivative_wrt_e(walker_results)

In [25]:
min_interval_keys = []

for i in range(len(derivatives) - 1):
    min_diff = np.inf
    interval_index = np.inf
    min_key = np.inf
    for j in range(i+1, len(derivatives)):
        overlapping_keys = list(set(derivatives[i].keys()).intersection(set(derivatives[j].keys())))

        if len(overlapping_keys) != 0:
            for key in overlapping_keys:
                diff_deriv = np.abs(derivatives[j][key] - derivatives[i][key])

                if diff_deriv < min_diff:
                    min_diff = diff_deriv
                    interval_index = j
                    min_key = key

    min_interval_keys.append([interval_index, min_key])

print(min_interval_keys)

[[2, -16], [2, -12], [5, -8], [5, -8], [6, -4], [8, 0], [8, 4], [9, 4], [9, 8]]


In [26]:
real_stitching_keys = []

for i in range(len(min_interval_keys)):
    key_interval_i = min_interval_keys[i][1]

    check_intersection = True

    for j in range(i+1, len(min_interval_keys)):
        if min_interval_keys[j][1] <= key_interval_i:
            check_intersection = False

    if check_intersection:
        real_stitching_keys.append(min_interval_keys[i])

In [ ]:
for i in range(len(real_stitching_keys)):
    if (i == 0):
        current_interval = walker_results[0]
        next_interval = walker_results[real_stitching_keys[0]]

        # Concat at
        concat = real_stitching_keys[i]
    else:
        current_interval = real_stitching_keys[i-1]
        next_interval = real_stitching_keys[i]

        concat = real_stitching_keys[i]

In [27]:
real_stitching_keys

[[2, -16], [2, -12], [5, -8], [6, -4], [8, 0], [9, 4], [9, 8]]

### Step by step

In [10]:
seed = 1001

results_logG = get_seed_and_dicts(f"../results_get_logG/periodic/prob_{probabilities:.6f}/X_{size}_Y_{size}/error_class_X/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt")

In [11]:
results_rescale_min = get_seed_and_dicts(f"../results_rescale_minimum/periodic/prob_{probabilities:.6f}/X_{size}_Y_{size}/error_class_X/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt")

In [21]:
rescaled_intervals = get_seed_and_dicts(f"../results_rescale_for_concat/periodic/prob_{probabilities:.6f}/X_{size}_Y_{size}/error_class_X/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt")

In [28]:
seed = 1000

end_results = get_seed_and_dicts(f"../results_end/periodic/prob_{probabilities:.6f}/X_{size}_Y_{size}/error_class_X/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt")

In [36]:
end_results

{1001: [{-24.0: 2.0093885885,
   -20.0: 4.0973564611,
   -16.0: 5.9367450701,
   -12.0: 7.4793981539,
   -8.0: 8.7596904264,
   -4.0: 9.5917784201,
   0.0: 9.8525468336,
   4.0: 9.5843664633,
   8.0: 8.7781111227,
   12.0: 7.5048545824,
   16.0: 5.9480921255,
   20.0: 4.1239861475,
   24.0: 2.0454904066}],
 1000: [{-24.0: 0.6110980203,
   -20.0: 3.3074729135,
   -16.0: 5.8848151853,
   -12.0: 7.6052999189,
   -8.0: 8.7890288522,
   -4.0: 9.5545742204,
   0.0: 9.8409199407,
   4.0: 9.5899986913,
   8.0: 8.8021990945,
   12.0: 7.6159920385,
   16.0: 5.8733665635,
   20.0: 3.2664317777,
   24.0: 0.5816769292}]}

In [43]:
for res in end_results:
    #print(end_results[res][0].values())
    print(np.log(2)*16 - log_sum_exp(end_results[res][0].values()))

-0.000000000015821780041608506556728428327384578650604445483213
-0.000000000014667257349131292203367175832101680697259658507381


## After logG

In [12]:
seed = 1001
results_old = read_data_from_file(f"../results/periodic_old_result/prob_0.100000/X_4_Y_4/seed_{seed}/error_class_X/intervals_10_iterations_1000_overlap_0.250000_walkers_8_seed_run_42_alpha_0.800000_beta_0.0000010000exchange_offset50.txt")

results_logG = get_seed_and_dicts(f"../results_get_logG/periodic/prob_{probabilities:.6f}/X_{size}_Y_{size}/error_class_X/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt")

In [14]:
diffs = []

for d in range(len(results_old)):
    interval_id = int(d/8)

    diff = [results_old[d][key] - results_logG[1001][interval_id][key] for key in results_old[d]]

    if np.max(np.abs(diff)) != 0:
        diffs.append((d,diff))

print(diffs)

[]


### Min rescaling

In [15]:
results_rescale_min = get_seed_and_dicts(f"../results_rescale_minimum/periodic/prob_{probabilities:.6f}/X_{size}_Y_{size}/error_class_X/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt")

In [16]:
"""averages over walker results per intervals"""
walker_results = average_matching_keys(results_old)

"""normalize the walker results by min value for log results"""
walker_results = get_renormalized_log_g_values_as_dict_list(walker_results)

In [17]:
diffs = []

for d in range(len(walker_results)):

    diff = [walker_results[d][key] - results_rescale_min[1001][d][key] for key in walker_results[d]]

    if np.max(np.abs(diff)) > 1e-9:
        diffs.append((d,diff))

print(diffs)

[]


### After rescale for concatenation

In [18]:
boundary_type = "periodic"
probabilities = 0.1
size = 4
intervals = 10
iterations = 1000
overlap = 0.25
walkers = 8
alpha = 0.8
beta = 1e-6
exchange = 50

rescaled_intervals = get_seed_and_dicts(f"../results_rescale_for_concat/periodic/prob_{probabilities:.6f}/X_{size}_Y_{size}/error_class_X/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt")

In [19]:
rescaled_intervals

{1001: [{-24.0: 0.0,
   -20.0: 2.0879678726,
   -16.0: 3.9273564816,
   -12.0: 5.4641010761},
  {-20.0: 2.1059215069,
   -16.0: 3.9273564816,
   -12.0: 5.4629769325,
   -8.0: 6.7276535034},
  {-16.0: 3.9273564816,
   -12.0: 5.4700095654,
   -8.0: 6.721883297,
   -4.0: 7.5087661743},
  {-12.0: 0.0, -8.0: 1.2792851925, -4.0: 2.0746188164},
  {-12.0: 5.4415910244,
   -8.0: 6.721883297,
   -4.0: 7.5539712906,
   0.0: 7.8297727108},
  {-8.0: 1.2640581131,
   -4.0: 2.0746188164,
   0.0: 2.3433334827,
   4.0: 2.1096146107},
  {-4.0: 7.5539712906,
   0.0: 7.8147397041,
   4.0: 7.5465593338,
   8.0: 6.7278399467},
  {0.0: 1.0414922237, 4.0: 0.7950513363, 8.0: 0.0},
  {0.0: 2.3167402744, 4.0: 2.0341844559, 8.0: 1.2480275631, 12.0: 0.0},
  {4.0: 2.1096146107,
   8.0: 1.3033592701,
   12.0: 0.0301027298,
   16.0: -1.5266597271,
   20.0: -3.3507657051,
   24.0: -5.429261446}]}

In [20]:
seed = 1001
results_old = read_data_from_file(f"../results/periodic_old_result/prob_0.100000/X_4_Y_4/seed_{seed}/error_class_X/intervals_10_iterations_1000_overlap_0.250000_walkers_8_seed_run_42_alpha_0.800000_beta_0.0000010000exchange_offset50.txt")

"""averages over walker results per intervals"""
walker_results = average_matching_keys(results_old)

"""normalize the walker results by min value for log results"""
walker_results = get_renormalized_log_g_values_as_dict_list(walker_results)

results_x = []
results_y = []
for result in walker_results:
    results_y.append(np.array(list(result.values())))
    results_x.append(np.array(list(result.keys())))

derivatives_wrt_e = get_derivative_wrt_e(walker_results)
minimum_deviation_energies = find_lowest_inverse_temp_deviation(derivatives_wrt_e)
rescale_results_for_concatenation(results_x, results_y, minimum_deviation_energies)

In [21]:
diffs=[]
for i in range(len(results_x)):
        diff = [results_y[i][j] - list(rescaled_intervals[seed][i].values())[j] for j in range(len(results_x[i]))]
        diffs.append(diff)

In [22]:
for d in diffs:
    print(np.max(np.abs(d)))

9.992007221626409e-14
1.0005063444396001e-10
1.000231009129493e-10
5.442598104500064
0.0010070801000665597
5.480359554400156
0.0010070802000745616
6.774254560600127
5.535121441000096
5.459691286199982


### Cut overlapping

In [40]:
cut_overlap = get_seed_and_dicts(f"../results_cut_overlapping/periodic/prob_{probabilities:.6f}/X_{size}_Y_{size}/error_class_X/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt")

In [44]:
cut_overlap

{1018: [{-24.0: 0.0,
   -20.0: 1.0793597698,
   -16.0: 3.2492690086,
   -12.0: 4.8574972153,
   -8.0: 6.2191202641,
   -4.0: 6.9108543396,
   0.0: 7.3210532665,
   4.0: 6.9426600933,
   8.0: 6.3018901348,
   12.0: 4.91994524,
   16.0: 3.3055503368,
   20.0: 1.1417651176,
   24.0: 0.0349338055}]}

In [43]:
seed = 1018
results_old = read_data_from_file(f"../results/periodic_old_result/prob_0.100000/X_4_Y_4/seed_{seed}/error_class_X/intervals_10_iterations_1000_overlap_0.250000_walkers_8_seed_run_42_alpha_0.800000_beta_0.0000010000exchange_offset50.txt")

"""normalize the walker results by min value for log results"""
walker_results = get_renormalized_log_g_values_as_dict_list(results_old)

"""averages over walker results per intervals"""
walker_results = average_matching_keys(walker_results)

results_x = []
results_y = []
for result in walker_results:
    results_y.append(np.array(list(result.values())))
    results_x.append(np.array(list(result.keys())))

derivatives_wrt_e = get_derivative_wrt_e(walker_results)
minimum_deviation_energies = find_lowest_inverse_temp_deviation(derivatives_wrt_e)
rescale_results_for_concatenation(results_x, results_y, minimum_deviation_energies)

"""Store concatenate interval results"""
concatenated_keys = np.concatenate(results_x)
concatenated_values = np.concatenate(results_y)
list_of_concat_rescale_dicts = []
for keys, values in zip(results_x, results_y):
    # Combine keys and values into a dictionary
    dict_from_arrays = {k: v for k, v in zip(keys, values)}
    list_of_concat_rescale_dicts.append(dict_from_arrays)

cut_overlapping_histogram_parts(list_of_concat_rescale_dicts, minimum_deviation_energies)